<a href="https://colab.research.google.com/github/GabyLeon10100/MujeresDigitales/blob/main/Taller_semana_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicción de pagos a préstamos

## Introducción

**Contexto comercial.** Como analista de un banco se le entrega un conjunto de datos con información de los clientes y si pagaron el crédito o no. Su jefe le pida que entrene algunos modelos para responder la pregunta: si un usuario incurrirá en incumplimiento o no. El algoritmo para predecir la probabilidad de incumplimiento es el núcleo del negocio.

**Problema comercial.** Su tarea es **crear un modelo de clasificación para determinar si un usuario incumplirá con su préstamo o no**.

**Contexto analítico.** Como científico de datos, ahora debe concentrarse en perfeccionar el algoritmo de clasificación. Debe verificar cuantitativamente que el algoritmo funcionará para datos futuros. Además, debe expandir su conjunto de herramientas para no depender únicamente de la regresión logística, ya que tiene algunas deficiencias que se manejan mejor con otros tipos de modelos. 

## Evaluar los diferentes modelos

In [3]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.model_selection import RandomizedSearchCV

In [4]:
#Traemos el archivo "pokemon_data.csv"
from google.colab import files
uploaded = files.upload()

Saving Lending_club_cleaned_2.csv to Lending_club_cleaned_2 (1).csv


In [5]:
df = pd.read_csv("Lending_club_cleaned_2.csv")

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
df.head()

,loan_status,annual_inc,verification_status,emp_length,home_ownership,int_rate,loan_amnt,purpose,term,grade
0,Fully Paid,24000.0,Verified,10+ years,RENT,10.65%,5000,credit_card,36 months,B
1,Charged Off,30000.0,Source Verified,< 1 year,RENT,15.27%,2500,car,60 months,C
2,Fully Paid,12252.0,Not Verified,10+ years,RENT,15.96%,2400,small_business,36 months,C
3,Fully Paid,49200.0,Source Verified,10+ years,RENT,13.49%,10000,other,36 months,C
4,Fully Paid,80000.0,Source Verified,1 year,RENT,12.69%,3000,other,60 months,B


## Ejercicio 1

Realice una estandarización de los datos númericos usando el promedio y la desviación estándar esto es: $(datos - promedio) / desviación\ estándar$, para las columnas ```[loan_amnt, int_rate, annual_inc]```. Y para las variables categóricas use one hot encoding ```[home_ownership, purpose, grade, loan_status]```.

Tip: Para las variables categóricas recuerde que puede usar la función de pandas: get_dummies

In [8]:
# Estandarización para la columna loan_amnt
df['loan_amnt'] = abs(df['loan_amnt'] - df['loan_amnt'].mean()) / df['loan_amnt'].std()
# Estandarización para la columna annual_inc
df['annual_inc'] = abs(df['annual_inc'] - df['annual_inc'].mean()) / df['annual_inc'].std()
# Estandarización para la columna int_rate
df['int_rate'] = df['int_rate'].str.replace('%', '').astype(float)
df['int_rate'] = abs(df['int_rate'] - df['int_rate'].mean()) / df['int_rate'].std()

In [9]:
#Estandarización con la función get_dummies
df = pd.get_dummies(df, columns=['home_ownership'], drop_first=False)
df = pd.get_dummies(df, columns=['purpose'], drop_first=False)
df = pd.get_dummies(df, columns=['grade'], drop_first=False)
df = pd.get_dummies(df, columns=['loan_status'], drop_first=False)

In [24]:
#Mostramos el df
df.head()

,annual_inc,verification_status,emp_length,int_rate,loan_amnt,term,home_ownership_MORTGAGE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,loan_status_Charged Off,loan_status_Fully Paid
0,0.710290,Verified,10+ years,0.378954,0.843862,36 months,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
1,0.616866,Source Verified,< 1 year,0.863143,1.178519,60 months,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2,0.893213,Not Verified,10+ years,1.048651,1.191906,36 months,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1
3,0.317912,Source Verified,10+ years,0.384586,0.174546,36 months,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1
4,0.161661,Source Verified,1 year,0.169505,1.111588,60 months,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1


## Ejercicio 1.1:

Realice una separación de los datos: entrenamiento 80% y prueba 20%

In [21]:
X = df.drop(['annual_inc'], axis=1)
y = df[['annual_inc']].values.ravel()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.80, random_state=0)

## Ejercicio 2:

Entrene 3 modelos para predecir si un cliente pagará el crédito o no:
1. Regresión logística
2. Árbol aleatorio (random forest)
3. KNN (vecinos mas cercanos)

In [ ]:
# Regresión logística


## Ejercicio 3:

Compare los tres modelos anteriores usando la métrica precisión. Recuerde que la fórmula es: $VP/(VP+FP)$

Donde:


VP: Verdaderos positivos

FP: Falsos positivos

In [ ]:
# Escriba aquí su código


## Ejercicio 4: (Opcional)

Usando la función `RandomizedSearchCV` busque el número de vecinos mas cercanos (k) que mejor prediga el conjunto de datos. Use los siguientes valores: `n_neighbors: entre 1 y 8`, `cross validation: 10` y ``scoring: 'precision'``

In [ ]:
# Escriba aquí su código


## Ejercicio 5: (Opcional)

Extraiga el valor de precisión del modelo optimizado en el ejercicio 4.

In [ ]:
# Escriba aquí su código
